## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of four are not allowed and you are more than welcome to form groups of two.
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
If a topic was not covered in detail during the lecture, you are asked to study it online on your own. 
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

For your convenience, , the code has a **DEBUG** mode that you may use in order to debug with toy data.  
It is recommended to solve the code in that mode (with efficiency in mind) and then run the code on all the data.
**Do not forget to file the HW with DEBUG == False**.

`Download the data` from [HERE](https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics) and put it in the same directory your script is.

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image

from IPython.core.display import HTML 
from pathlib import Path

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

# Debug
""" you can change this cell """

In [ ]:
DEBUG = False
"""
Recommended to start with a small number to get a feeling for the preprocessing with prints (N_ROWS_FOR_DEBUG = 2)
later increase this number for 5*10**3 in order to see that the code runs at reasonable speed. 
When setting Debug == False, our code implements bow.fit() in 15-20 minutes according to the tqdm progress bar. Your solution is not supposed to be much further than that.
"""
N_ROWS_FOR_DEBUG = 5*10**3 

# Config

In [ ]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
CHUNCK_SIZE = 5 if DEBUG else 5*10**3
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [ ]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [ ]:
def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):
        word = word.lower()
        if word in stop_words:
            continue
        for char in word:
            if char not in allowed_symbols:
                word = word.replace(char, '')
        word = stemmer.stem(word)
        if len(word) <= 1:
            continue
        output_sentence.append(word)
    return output_sentence


def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))]

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            self.unigram_count[word] += 1
            if word in self.inverted_index:
                if document_id in self.inverted_index[word]:
                    self.inverted_index[word][document_id] += 1
                else:
                    self.inverted_index[word][document_id] = 1
            else:
                self.inverted_index.update({word:{document_id: 1}})
            if i < sentence_len - 1:
                bigram = word + "," + sentence[i+1]
                self.bigram_count[bigram] += 1
                if i < sentence_len - 2:
                    trigram = bigram + "," +sentence[i+2]
                    self.trigram_count[trigram] +=1


    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck:
                self.n_docs += 1
                self.doc_norms[self.n_docs] = 0
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()

    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():
            self.word_document_frequency[word] = len(self.inverted_index[word])



    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        for word in self.inverted_index.keys():
            for doc in self.inverted_index[word].keys():
                tf = self.inverted_index[word][doc] / self.document_term_frequency[doc]
                idf = np.log10(self.n_docs/self.word_document_frequency[word])
                self.inverted_index[word][doc] = tf*idf
                self.doc_norms[doc] += np.square(self.inverted_index[word][doc])
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc])

    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)

tf_idf = TfIdf()
tf_idf.fit()


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [ ]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms

    def rank(self,query : Dict[str,int],documents: List[Counter],metric: str ) -> Dict[str, float]:
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        for word in query:
            query[word]=query[word]/query_len*np.log10(self.n_docs/self.word_document_frequency[word])    
            for doc in documents[word]:
                result[doc]=result.get(doc,0)+query[word]*self.inverted_index[word][doc]
        if metric == 'cosine':
            for doc in result.keys():
                result[doc]=result[doc]/self.doc_norms[doc]

        return result

    def sort_and_retrieve_k_best(self, scores: Dict[str, float],k :int):
        sorted_list_keys =  {key : v for key, v in sorted(scores.items(), key = lambda item: item[1], reverse=True)[:k]}

        return list(sorted_list_keys.keys())

    def reduce_query_to_counts(self, query : List)->  Counter:
        query_counts = Counter()
        for i, word in enumerate(query):
            query_counts[word] += 1
        return query_counts



    def get_top_k_documents(self,query : str, metric: str , k = 5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)

dr = DocumentRetriever(tf_idf)

In [ ]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [ ]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

In [ ]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

# 1.3 term statistics:
Use "bow" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [ ]:
# 1. 
### YOUR SOLUTION HERE
print('unigram_count: ' + str(len(tf_idf.unigram_count)))
### END YOUR SOLUTION

"""
### Your verbal solution here
The tf_idf.unigram_count Counter holds all of the unique words in our vocabulary. 
### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE
print('potential : ' + str((len(tf_idf.unigram_count))*(len(tf_idf.unigram_count))))
print('bigram_count: ' + str(len(tf_idf.bigram_count)))
### END YOUR SOLUTION

"""
### Your verbal solution here
The number of potenial bigrams is the square of the number of unique words in our corpus, which is: 150750486756
In reality we only have 6308457 bigrams.
The difference might happen because not all pairs are viable by grammar rules, logic rules or just less likely to be paired
to one another in songs.

### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE
import os
print("lyrics storage size: ",Path(INPUT_FILE_PATH).stat().st_size/(1024*1024))
print("bow storage size: ",Path(tf_idf.bow_path).stat().st_size/(1024*1024))
### END YOUR SOLUTION

"""
### Your verbal solution here
Size of the lyrics file is about ~315MB on disc and the size of the bow file is about ~190MB on disc.

The difference can be explained by the three steps we took in order to create the bow file:
1. Out of the whole file we started with we exclusively used only the lyrics part.
2. In the preprocessing stage we removed all of the stop words, not allowed symbols and stemmed all words, ending up with 
processed sentences that are smaller and lighter than the ones we started with.
3. In the "update_counts_and_probablities", we created the inverted index which was later saved as the bow file.
That object is a dictionary containing the different words, documents and number of times they appeared in each combination.
So no matter how many times the word repeated itself in a song within the original file, the bow file contains only the word itself 
and its number of appearences in a document, resulting in another big reduction in sizes.
 
### End your verbal solution here
"""

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [ ]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))

def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))


"""
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

In [ ]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram

    def build_index(self) -> None:
        for word in self.unigram_counts.keys():
            for ngram in self.get_n_grams(word):
                if self.ngram_index.get(ngram):
                    self.ngram_index[ngram].append(word)
                else:
                    self.ngram_index[ngram] = [word]


    def get_top_k_words(self,word:str,k=5) -> List[str]:
        total_words = sum(self.unigram_counts.values())
        list_x = [ngram for ngram in self.get_n_grams(word)]
        similarity_dict = {}
        for ngram in list_x:
            if self.ngram_index.get(ngram):
                for real_word in self.ngram_index.get(ngram):
                    if not similarity_dict.get(real_word):
                        prior = self.unigram_counts[real_word] / total_words
                        list_y = [ngram for ngram in self.get_n_grams(real_word)]
                        similarity_dict[real_word] = prior * jaccard_similarity(list_x, list_y)

        return sorted(similarity_dict.keys(),key=lambda item : similarity_dict[item],reverse=True)[:k]


def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)


class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)#%%

In [ ]:
out_of_vocab_word = 'supercalifragilisticexpialidocious'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)


In [ ]:

trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  p(does|spiderman) \cdot p(whatever|does) \cdot  p(a|whatever) \cdot  p(spider|a) \cdot p(can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  p(whatever|spiderman,does) \cdot p(a|does,whatever) \cdot  p(spider|whatever,a) \cdot  p(can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [ ]:
# for the probability smoothing
NUMERATOR_SMOOTHING = 1 # alpha in https://en.wikipedia.org/wiki/Additive_smoothing
DENOMINATOR_SMOOTHING = 10**4 # d in https://en.wikipedia.org/wiki/Additive_smoothing
def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0
    words_in_sentence = sentence.split()
    n_words = len(words_in_sentence)
    for index, word in  enumerate(words_in_sentence):
        if index == 0:
            bigram_log_likelilhood += np.log((unigrams[word] + NUMERATOR_SMOOTHING) / (sum(unigrams.values()) + DENOMINATOR_SMOOTHING))
        elif index == 1:
            bigram_log_likelilhood += np.log(((bigrams[words_in_sentence[index-1]+","+word]  / sum(bigrams.values())) + NUMERATOR_SMOOTHING)/
                                             ((unigrams[words_in_sentence[index-1]] / sum(unigrams.values())) + DENOMINATOR_SMOOTHING))
            trigram_log_likelilhood += np.log((bigrams[words_in_sentence[index-1]+","+word] + NUMERATOR_SMOOTHING) / (sum(bigrams.values()) + DENOMINATOR_SMOOTHING))
        else:
            bigram_log_likelilhood += np.log(((bigrams[words_in_sentence[index-1]+","+word]  / sum(bigrams.values())) + NUMERATOR_SMOOTHING)/
                                              ((unigrams[words_in_sentence[index-1]] / sum(unigrams.values())) + DENOMINATOR_SMOOTHING))
            trigram_log_likelilhood += np.log(((trigrams[words_in_sentence[index-2]+","+words_in_sentence[index-1]+","+word] / sum(trigrams.values()))+ NUMERATOR_SMOOTHING) /
                                              ((bigrams[words_in_sentence[index-2]+","+words_in_sentence[index-1]] / sum(bigrams.values())) + DENOMINATOR_SMOOTHING))
     
    print(F"Bigram log likelihood is {bigram_log_likelilhood}")
    print(F"Trigram log likelihood is {trigram_log_likelilhood}")
    
sentence = "spider man spider man does whatever a spider can"
sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)

## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "i am"?

In [ ]:
### YOUR CODE HERE
sentence_to_predict = "i am"

bigram_list = [bigram for bigram in tf_idf.bigram_count.keys() if str(bigram).startswith(sentence_to_predict.split()[-1] + ",")]
trigram_list = [trigram for trigram in tf_idf.trigram_count.keys() if str(trigram).startswith(sentence_to_predict.split()[-2] + "," + sentence_to_predict.split()[-1] + ",")]

dict_bigram = {bigram : np.log((tf_idf.bigram_count[bigram] + NUMERATOR_SMOOTHING) /(tf_idf.unigram_count[bigram.split(',')[0]] + NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)) for bigram in bigram_list}
if (bool(dict_bigram)): next_bigram_word = max(dict_bigram, key=dict_bigram.get)
else: next_bigram_word = max(tf_idf.unigram_count, key=tf_idf.unigram_count.get)


dict_trigram = {trigram : np.log((tf_idf.trigram_count[trigram] + NUMERATOR_SMOOTHING) / (tf_idf.bigram_count[trigram.split(',')[0]+","+trigram.split(',')[1]] + NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)) for trigram in trigram_list}
if (bool(dict_trigram)): next_trigram_word = max(dict_trigram, key=dict_trigram.get)
else: next_trigram_word = max(tf_idf.unigram_count, key=tf_idf.unigram_count.get)


print("Next word predicition according to bigram model is: " + next_bigram_word.split(',')[-1])
print("The full sentece is: " + sentence_to_predict + " " + next_bigram_word.split(',')[-1])
print("Next word predicition according to trigram model is: " + next_trigram_word.split(',')[-1])
print("The full sentece is: " + sentence_to_predict + " " + next_trigram_word.split(',')[-1])

### END YOUR CODE

In [ ]:
'''
Our idea for predicting the next word according to the bigram/trigram model, was extracting all exisiting bigrams/trigrams
that end in a similar way to the sentence we're trying to predict on.
ie. for "i am" we'll look for any bigram that looks like "am xxx" (am mess, am not, etc)
and any trigram that looks like "i am xxx" (i am creep, i am weirdo) 
and then from those selected bigrams/trigrams, find the one that is the most likely to appear by relying on our data.
if no bigram/trigram exist, we will use the most common word from our unigram list.
'''
